# Часть 1

1) Извлечем zip архив

In [31]:
import zipfile

def extract_encrypted_zip(zip_file_path, password, destination_path):
    try:
        with zipfile.ZipFile(zip_file_path) as zip_file:
            zip_file.setpassword(bytes(password, 'utf-8'))
            zip_file.extractall(path=destination_path)
            print("Файлы успешно извлечены")
    except:
        print(f"Error: Произошла ошибка во время извлечения файла")

In [32]:
zip_file_path = 'protected_archive.zip'
file_name = 'invoice-42369643.html'
destination_path = 'extracted_files/'
password = 'netology'
extract_encrypted_zip(zip_file_path, password, destination_path)

Файлы успешно извлечены


2) Отправим файл в VirusTotal

In [3]:
api_key = input()

In [4]:
import requests

def send_to_virus_total(file_path):
    api_url = "https://www.virustotal.com/api/v3/files"
    headers = {"x-apikey" : api_key}
    with open(file_path, "rb") as file:
        files = {"file": (file_path, file)}
        response = requests.post(api_url, headers=headers, files=files)
        return response.json()

In [5]:
file_path = 'extracted_files/invoice-42369643.html'
vt_response = send_to_virus_total(file_path)
analysis_id = vt_response['data']['id']

3) Посмотрим на детальный анализ данного файла

In [33]:
import time
def get_analysis_info(analysis_id):
    headers = {"x-apikey" : api_key}
    url_analysis_details = f'https://www.virustotal.com/api/v3/analyses/{analysis_id}'
    response_analysis = requests.get(url_analysis_details, headers=headers)
    time.sleep(30)
    if response_analysis.status_code == 200:
        analysis_details = response_analysis.json()
        return analysis_details
    else:
        print('Ошибка при получении деталей анализа:', response_analysis.text)
        return ""
analysis_info = get_analysis_info(analysis_id)

In [39]:
analysis_info

{'data': {'id': 'NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0Mjc5OTY4MQ==',
  'type': 'analysis',
  'links': {'self': 'https://www.virustotal.com/api/v3/analyses/NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0Mjc5OTY4MQ==',
   'item': 'https://www.virustotal.com/api/v3/files/f4dc2e1abf2de55b455c7dfc0bbe66a9d12f7e857cd12f8408dadd7129946ae3'},
  'attributes': {'stats': {'malicious': 20,
    'suspicious': 0,
    'undetected': 34,
    'harmless': 0,
    'timeout': 8,
    'confirmed-timeout': 0,
    'failure': 0,
    'type-unsupported': 15},
   'status': 'completed',
   'results': {'Bkav': {'method': 'blacklist',
     'engine_name': 'Bkav',
     'engine_version': '2.0.0.1',
     'engine_update': '20250324',
     'category': 'undetected',
     'result': None},
    'Lionic': {'method': 'blacklist',
     'engine_name': 'Lionic',
     'engine_version': '8.16',
     'engine_update': '20250324',
     'category': 'malicious',
     'result': 'Trojan.Script.Obfuse.4!c'},
    'MicroWorld-eScan':

4) Выведем список угроз, которые мы обнаружили

In [34]:
print("Detected:")
for values in analysis_info['data']['attributes']['results'].values():
    if 'category' in values and values['category'] == 'malicious':
        print(values['engine_name'], values['result'])

Detected:
Lionic Trojan.Script.Obfuse.4!c
MicroWorld-eScan Trojan.Generic.37575014
CTX txt.trojan.obfuse
ALYac Trojan.Generic.37575014
Symantec Trojan.Gen.MBT
ESET-NOD32 VBA/TrojanDropper.Agent.AXB
Kaspersky UDS:DangerousObject.Multi.Generic
Arcabit Trojan.Generic.D23D5966
Emsisoft Trojan.Generic.37575014 (B)
DrWeb X97M.DownLoader.459
VIPRE Trojan.Generic.37575014
FireEye Trojan.Generic.37575014
Kingsoft Win32.Infected.AutoInfector.a
Microsoft Trojan:O97M/Obfuse.CP
ViRobot HTML.Z.Agent.48950
GData Trojan.Generic.37575014
Google Detected
Ikarus Trojan-Dropper.VBA.Agent
Fortinet VBA/Agent.BCF0!tr
alibabacloud Trojan[dropper]:MSOffice/Obfuse.CX


# Часть 2

1) Десериализуем данный json

In [35]:
import json

json_data = '''
[
    {
        "Program": "LibreOffice",
        "Version": "6.0.7"
    },
    {
        "Program": "7zip",
        "Version": "18.05"
    },
    {
        "Program": "Adobe Reader",
        "Version": "2018.011.20035"
    },
    {
        "Program": "nginx",
        "Version": "1.14.0"
    },
    {
        "Program": "Apache HTTP Server",
        "Version": "2.4.29"
    },
    {
        "Program": "DjVu Reader",
        "Version": "2.0.0.27"
    },
    {
        "Program": "Wireshark",
        "Version": "2.6.1"
    },
    {
        "Program": "Notepad++",
        "Version": "7.5.6"
    },
    {
        "Program": "Google Chrome",
        "Version": "68.0.3440.106"
    },
    {
        "Program": "Mozilla Firefox",
        "Version": "61.0.1"
    }
]
'''

data = json.loads(json_data)

In [36]:
vulners_api_key = input()

2) Получим список уязвимостей ПО, используя API Vulners

In [ ]:
import vulners

vulners_api = vulners.Vulners(api_key=vulners_api_key)

program_version = []
for element in data:
    program_version.append({
        "product": element['Program'],
        "version": element['Version']
    })

response = vulners_api.audit_software(
    software=program_version,
    fields=["title", "short_description"],
    match='partial'
)

response

3) Получим список ПО-уязвимости

In [42]:
for item in response:
    print(f"Name: {item['input']['product']}, cve amount:{len(item['vulnerabilities'])}")

Name: Adobe Reader, cve amount:0
Name: LibreOffice, cve amount:8
Name: Wireshark, cve amount:52
Name: Apache HTTP Server, cve amount:76
Name: Notepad++, cve amount:0
Name: nginx, cve amount:24
Name: Mozilla Firefox, cve amount:1144
Name: Google Chrome, cve amount:2867


In [ ]:
for item in response:
    print(f"Name: {item['input']['product']}, cve amount:{len(item['vulnerabilities'])}")
    for vuln in item['vulnerabilities']:
        print(vuln['id'])